In [ ]:
import numpy as np
import pandas as pd
import os
import sys
root_dir = os.path.dirname(os.path.dirname(os.getcwd())) ## TODO: find way to put this into some global settings
if root_dir not in sys.path:
    sys.path.append(root_dir)

from references import common_cfg, istat_kpi

In [ ]:
farmacieRaw = pd.read_csv('../../data/raw/health/farmacie.csv', low_memory=False, sep=';', decimal=',')

In [ ]:
farmacie = farmacieRaw[farmacieRaw.DATAFINEVALIDITA == '-']
farmacie.set_index('CODICEIDENTIFICATIVOFARMACIA', inplace=True)
assert not any(farmacie.index.duplicated()), 'Duplicates in pharmacy ids'
# rename location columns
farmacie = farmacie.rename(columns=dict(zip(['LONGITUDINE','LATITUDINE'], common_cfg.coord_col_names)))

In [ ]:
farmacie.columns

In [ ]:
# drop missing coordinates
print(farmacie[farmacie.Long == '-'].DESCRIZIONECOMUNE.value_counts())
farmacie = farmacie[~(farmacie.Long == '-')]  # do not reset index

In [ ]:
# convert the valid ones to floats
farmacie[common_cfg.coord_col_names] = farmacie[common_cfg.coord_col_names].astype(float)
assert all(farmacie.Lat > 0) & all(farmacie.Long > 0), 'Unexpected values in coordinates'

In [ ]:
for cittaScelta in common_cfg.city_list:
    farmacieCitta = farmacie[farmacie.DESCRIZIONECOMUNE.apply(str.lower) == cittaScelta.lower()]
    # save to csv
    farmacieCitta.to_csv('../../data/processed/%s_farmacie.csv' % cittaScelta, sep=';', decimal = '.') 